---
layout: post

title: 백준 문제 중 14502번 with python

category: Algorithms
---


# 백준 문제 중 14502번
<a>https://www.acmicpc.net/problem/14502<a/>

### 문제

인체에 치명적인 바이러스를 연구하던 연구소에서 바이러스가 유출되었다. 다행히 바이러스는 아직 퍼지지 않았고, 바이러스의 확산을 막기 위해서 연구소에 벽을 세우려고 한다.

연구소는 크기가 N×M인 직사각형으로 나타낼 수 있으며, 직사각형은 1×1 크기의 정사각형으로 나누어져 있다. 연구소는 빈 칸, 벽으로 이루어져 있으며, 벽은 칸 하나를 가득 차지한다. 

일부 칸은 바이러스가 존재하며, 이 바이러스는 상하좌우로 인접한 빈 칸으로 모두 퍼져나갈 수 있다. 새로 세울 수 있는 벽의 개수는 3개이며, 꼭 3개를 세워야 한다.

예를 들어, 아래와 같이 연구소가 생긴 경우를 살펴보자.

    2 0 0 0 1 1 0
    0 0 1 0 1 2 0
    0 1 1 0 1 0 0
    0 1 0 0 0 0 0
    0 0 0 0 0 1 1
    0 1 0 0 0 0 0
    0 1 0 0 0 0 0

이때, 0은 빈 칸, 1은 벽, 2는 바이러스가 있는 곳이다. 아무런 벽을 세우지 않는다면, 바이러스는 모든 빈 칸으로 퍼져나갈 수 있다.

2행 1열, 1행 2열, 4행 6열에 벽을 세운다면 지도의 모양은 아래와 같아지게 된다.

    2 1 0 0 1 1 0
    1 0 1 0 1 2 0
    0 1 1 0 1 0 0
    0 1 0 0 0 1 0
    0 0 0 0 0 1 1
    0 1 0 0 0 0 0
    0 1 0 0 0 0 0
바이러스가 퍼진 뒤의 모습은 아래와 같아진다.  

    2 1 0 0 1 1 2
    1 0 1 0 1 2 2
    0 1 1 0 1 2 2
    0 1 0 0 0 1 2
    0 0 0 0 0 1 1
    0 1 0 0 0 0 0
    0 1 0 0 0 0 0

벽을 3개 세운 뒤, 바이러스가 퍼질 수 없는 곳을 안전 영역이라고 한다. 위의 지도에서 안전 영역의 크기는 27이다.

연구소의 지도가 주어졌을 때 얻을 수 있는 안전 영역 크기의 최댓값을 구하는 프로그램을 작성하시오.

### 입력

첫째 줄에 지도의 세로 크기 N과 가로 크기 M이 주어진다. (3 ≤ N, M ≤ 8)

둘째 줄부터 N개의 줄에 지도의 모양이 주어진다. 0은 빈 칸, 1은 벽, 2는 바이러스가 있는 위치이다. 2의 개수는 2보다 크거나 같고, 10보다 작거나 같은 자연수이다.

빈 칸의 개수는 3개 이상이다.

### 출력

첫째 줄에 얻을 수 있는 안전 영역의 최대 크기를 출력한다.

---

3개의 벽을 최적의 방법으로 세우는 방법이 뭘까 고민하다가 그냥 모든 경우를 탐색해야 겠다는 생각이 들었다.

3개벽을 고르는 모든 경우는 dfs로도 구현이 가능하지만 간단하게 itertools 모듈을 사용했다.

코드의 흐름은 다음과같다.
1. 주어진 맵에서 각 바이러스를 viruses에 각 빈칸의 좌표를 blanks에 그리고 벽의 총 개수를 w_cnt에 저장한다.
2. 빈칸의 후보 중 itertools모듈로 3개를 골라 조합한뒤 candidates에 저장한다.
3. 감염된 곳의 수를 나타내는 infect, 지도를 나타내는 board를 변수로 갖고, 감염될 수 있는 모든 지역을 탐색할 bfs를 선언한다. 입력받은 infect보다 현재 맵에 바이러스가 많다면 후보가 될 수 없으므로 바로 리턴한다.
4. candidates에서 벽의 후보를 하나씩 골라 후보로 뽑힌 역을 벽으로 만든뒤 bfs를 호출해 최소 감염수 infect를 탐색하고 탐색을 마친뒤 벽을 다시 빈칸으로 만들어준다.
5. candidates의 모든 후보 탐색을 마치고 구한 최소 infect로 최대 안전구역을 구한다.

In [86]:
from itertools import combinations
from copy import deepcopy
from collections import deque
from pprint import pprint
from typing import List


n, m = map(int, input().split())
board = []
for _ in range(n):
    board.append(list(input().split()))

 # 바이러스, 빈칸을 각각 리스트에 저장
viruses, blanks = [], []
 # 벽의 총 개수 카운트
w_cnt = 3
for row in range(n):
    for col in range(m):
        if board[row][col] == '2':
            viruses.append([row, col])
        elif board[row][col] == '0':
            blanks.append([row, col])
        else:
            w_cnt += 1


 # 세울 수 있는 벽의 후보
candidates = combinations(blanks, 3)

dr, dc = [1, -1, 0, 0], [0, 0, 1, -1]

def bfs(infect:int, board:List[List[int]])->int:  
   
     # 바이러스 수를 셀 변수
    v_cnt = 0
    q = deque()
     # 초기 바이러스 위치를 큐에 넣음
    for virus in viruses:
        v_cnt += 1
        q.append(virus)

    while q:
         # 입력으로 받은 infect보다 v_cnt가 크다면 백트래킹
        if v_cnt > infect:
            return infect
        r, c = q.popleft()
        for i in range(4):
            nr, nc = r + dr[i], c + dc[i]
            if 0 <= nr < n and 0 <= nc <m and board[nr][nc] == '0':
                v_cnt += 1
                q.append([nr, nc])
                board[nr][nc] = '2'
    
    return v_cnt

infect = m*n
for candidate in candidates:
     # 후보 벽을 세움
    for x,y in candidate: board[x][y] = '1'
    infect = min(infect, bfs(infect, deepcopy(board)))
     # 후보 벽을 없애줌
    for x,y in candidate: board[x][y] = '0'


ans = m*n - infect - w_cnt
print(ans)

58
3
